In [2]:
from typing import Dict, List, Tuple, Optional
from copy import deepcopy
from environment import Environment, State, id_rc
from search import dijkstra

In [3]:
def reconstruct_path(prev: Dict[int, Optional[int]], start: int, goal: int, width: int) -> Optional[List[Tuple[int,int]]]:
    if prev.get(goal) is None and start != goal:
        return None
    path_ids = []
    cur = goal
    while cur is not None:
        path_ids.append(cur)
        if cur == start:
            break
        cur = prev[cur]
    path_ids.reverse()
    return [id_rc(n, width) for n in path_ids]

In [4]:
def compute_dist_matrix(env: Environment, relevant: List[int]) -> Tuple[Dict[int, Dict[int, Optional[float]]], Dict[int, Dict[int, Optional[List[Tuple[int,int]]]]]]:
    dist_matrix: Dict[int, Dict[int, Optional[float]]] = {}
    path_matrix: Dict[int, Dict[int, Optional[List[Tuple[int,int]]]]] = {}
    for u in relevant:
        dist, prev = dijkstra(env, env.base_id, u)
        dist_matrix[u] = {}
        path_matrix[u] = {}
        for v in relevant:
            if dist.get(v, float('inf')) == float('inf'):
                dist_matrix[u][v] = None
                path_matrix[u][v] = None
            else:
                dist_matrix[u][v] = dist[v]
                path_matrix[u][v] = reconstruct_path(prev, u, v, env.w)
    return dist_matrix, path_matrix

In [21]:
def get_candidate_nodes(env: 'Environment') -> List[int]:
    candidates = []
   
    for nid, node in env.nodes.items():
        if getattr(node, "terrain", "") == "plano":
            id=id_rc(nid, 5)
            candidates.append(id)
    return candidates


In [ ]:
if __name__ == "__main__":
    grid = [
        ['plano', 'arena', 'plano', 'rocas', 'plano'],
        ['plano', 'X',     'dunas', 'X',     'plano'],
        ['plano', 'arena', 'rocas', 'arena', 'plano'],
        ['plano', 'X',     'plano', 'X',     'plano'],
        ['plano', 'plano', 'plano', 'arena', 'plano']
    ]
    env=Environment(grid, cell_size_m=1.0)
    candidates=get_candidate_nodes(env)

    pool = get_candidate_nodes(env, terrain_filter=['POI'])
    print("POIs candidatos (ids):", pool)

    relevant = [env.base_id] + pool
    dist_matrix, path_matrix = compute_dist_matrix(env, relevant)

    init_state = State(pos=env.base_id, battery=25.0, memory=1300.0)

    selected_csp = csp_select_pois_by_battery_memory(
        base_id=env.base_id,
        candidates=pool,
        dist_matrix=dist_matrix,
        max_battery=15.0,
        action_cost=1.0,    
        max_memory=10.0
    )
    print("Selección CSP inicial:", selected_csp)

    best_ind, best_fit = ga_main(
        base_id=env.base_id,
        pool=pool,
        dist_matrix=dist_matrix,
        path_matrix=path_matrix,
        env=env,
        init_state=init_state,
        pop_size=30,
        gens=100,
        action_cost=1.0,
        transmit_cost=3.0,  
        safety_margin=0.5
    )
    print("Fitness mejor individuo:", best_fit)

    index_map = {p: i for i, p in enumerate(pool)}
    best_tour = decode(best_ind, pool, index_map)
    print("Tour propuesto (ids):", best_tour)

    feasible, final_state, profile, remaining_batt, total_cost = simulate(
        state=init_state,
        base_id=env.base_id,
        tour=best_tour,
        dist_matrix=dist_matrix,
        path_matrix=path_matrix,
        env=env,
        action_cost=1.0,
        transmit_cost=3.0,
        safety_margin=0.5
    )

    print("Factible:", feasible)
    print("Batería restante:", remaining_batt)
    print("Coste total:", total_cost)
    print("Ruta recorrida (ids):", final_state.route)
    print("Perfil de acciones:")
    for step in profile:
        print(" ", step)

    if feasible:
        cur = env.base_id
        full_coords = []
        for p in best_tour + [env.base_id]:
            seg = path_matrix.get(cur, {}).get(p)
            if seg:
                if not full_coords:
                    full_coords.extend(seg)
                else:
                    # evitar duplicar el nodo de unión
                    if full_coords[-1] == seg[0]:
                        full_coords.extend(seg[1:])
                    else:
                        full_coords.extend(seg)
            cur = p
        print("Camino completo en coordenadas:", full_coords)
